RTS from text files

In [1]:
from databaker.framework import *
import pandas as pd

In [2]:
%run fetch_sources.ipynb
new_table = pd.read_csv(input_file, delim_whitespace= True, header = None)

In [3]:
new_table_data = new_table[new_table[3].isnull() == False]

In [4]:
new_table = new_table[new_table[3].isnull() == True]

In [5]:
new_table_data.shape

(5331, 4)

In [6]:
new_table_data.head(2)

,0,1,2,3
0,4Q2017EEAAA,#1000,251,1.0
1,4Q2017EEAAA,#1001,1601,1851.0


In [7]:
new_table_data[1] =new_table_data[2]

In [8]:
new_table_data[2] =new_table_data[3]

In [9]:
new_table = pd.concat([new_table, new_table_data])

In [10]:
new_table.drop([3], axis =1, inplace =True)

In [11]:
new_table.shape

(103534, 3)

In [12]:
new_table.head(6)

,0,1,2
51,4Q2017EEAA800AU000,3086,19.0
52,4Q2017EEAA800AU001,23,5.0
53,4Q2017EEAA800AU002,0,0.0
54,4Q2017EEAA800AU004,1405,421.0
55,4Q2017EEAA800AU005,367,335.0
56,4Q2017EEAA800AU006,27,15.0


In [13]:
new_table['Period'] ="quarter/"+ new_table[0].astype(str).str[2:6]+'-'+new_table[0].astype(str).str[0:2].str[::-1]

In [14]:
new_table['Flow'] = new_table[0].astype(str).str[6:7]

In [15]:
new_table['nuts1'] = new_table[0].astype(str).str[7:9]

In [16]:
new_table['Geography'] = new_table[0].astype(str).str[9:10]

In [17]:
new_table['Codeseq'] = new_table[0].astype(str).str[10:13]

In [18]:
new_table['Codalpha'] = new_table[0].astype(str).str[13:15]

In [19]:
new_table['SITC Section'] = new_table[0].astype(str).str[15:16]

In [20]:
new_table['SITC Division'] = new_table[0].astype(str).str[16:18]

In [21]:
new_table.rename(index = str, columns = {2:'Mass'}, inplace = True)

In [22]:
new_table.rename(index = str, columns = {1:'Value'}, inplace = True)

In [23]:
# new_table['Unit'] = '£ 000s'
# new_table['Measure Type'] = 'GBP Total'
new_table['Unit'] = "Kg 000s"
new_table['Measure Type'] = 'Kgs Total'

In [24]:
new_table =new_table[['Geography','Period','nuts1','Codeseq','Codalpha','SITC Section','SITC Division','Flow','Measure Type','Mass','Unit']]

In [25]:
new_table['Flow'] = new_table['Flow'].map(lambda cell:cell.replace('E', 'Exports'))
new_table['Flow'] = new_table['Flow'].map(lambda cell:cell.replace('I', 'Imports'))

In [26]:
new_table['Flow'].unique()

array(['Exports', 'Imports'], dtype=object)

In [27]:
new_table['Geography'].unique()

array(['A', 'B', 'C', 'D', 'F', 'G', 'H', 'I', 'J', 'E'], dtype=object)

In [28]:
replacements = {
   'Geography': {
      r'(A)': 'Asia and Oceania',
       r'(B)': 'Eastern Europe',
       r'(C)': 'European Union',
       r'(D)': 'Latin America and the Caribbean',
       r'(E)': 'Low Value Trade (Non-EU)',
       r'(F)': 'Middle East and North Africa',
       r'(G)': 'North America',
       r'(H)': 'Sub-Saharan Africa',
       r'(I)': 'Western Europe',
       r'(J)': 'Ships and Stores(Non-EU)'
       }
}

new_table.replace(replacements, regex=True, inplace=True)


In [29]:
new_table.count()

Geography        103534
Period           103534
nuts1            103534
Codeseq          103534
Codalpha         103534
SITC Section     103534
SITC Division    103534
Flow             103534
Measure Type     103534
Mass             103534
Unit             103534
dtype: int64

In [30]:
def user_perc(x):
    
    if x.strip(' ') == '':
        return 'NA'
    else:
        return x
    
new_table['Codalpha'] = new_table.apply(lambda row: user_perc(row['Codalpha']), axis = 1)
new_table['SITC Section'] = new_table.apply(lambda row: user_perc(row['SITC Section']), axis = 1)
new_table['SITC Division'] = new_table.apply(lambda row: user_perc(row['SITC Division']), axis = 1)


In [31]:
new_table['Mass'].dtype

dtype('float64')

In [32]:
new_table['Mass'] = new_table['Mass'].astype('int', copy=False)

In [33]:
new_table = new_table[new_table['Mass'] != 0 ]

In [34]:
new_table.count()

Geography        74214
Period           74214
nuts1            74214
Codeseq          74214
Codalpha         74214
SITC Section     74214
SITC Division    74214
Flow             74214
Measure Type     74214
Mass             74214
Unit             74214
dtype: int64

In [35]:
new_table.tail(5)

,Geography,Period,nuts1,Codeseq,Codalpha,SITC Section,SITC Division,Flow,Measure Type,Mass,Unit
103275,Sub-Saharan Africa,quarter/2017-Q4,ZB,H,NA,NA,NA,Imports,Kgs Total,43,Kg 000s
103277,Sub-Saharan Africa,quarter/2017-Q4,ZB,H,NA,NA,NA,Imports,Kgs Total,5,Kg 000s
103278,Sub-Saharan Africa,quarter/2017-Q4,ZB,H,NA,NA,NA,Imports,Kgs Total,7,Kg 000s
103389,Western Europe,quarter/2017-Q4,ZB,I,NA,NA,NA,Imports,Kgs Total,5298,Kg 000s
103394,Western Europe,quarter/2017-Q4,ZB,I,NA,NA,NA,Imports,Kgs Total,3,Kg 000s


In [36]:
new_table.head(5)

,Geography,Period,nuts1,Codeseq,Codalpha,SITC Section,SITC Division,Flow,Measure Type,Mass,Unit
51,Asia and Oceania,quarter/2017-Q4,EA,800,AU,0,00,Exports,Kgs Total,19,Kg 000s
52,Asia and Oceania,quarter/2017-Q4,EA,800,AU,0,01,Exports,Kgs Total,5,Kg 000s
54,Asia and Oceania,quarter/2017-Q4,EA,800,AU,0,04,Exports,Kgs Total,421,Kg 000s
55,Asia and Oceania,quarter/2017-Q4,EA,800,AU,0,05,Exports,Kgs Total,335,Kg 000s
56,Asia and Oceania,quarter/2017-Q4,EA,800,AU,0,06,Exports,Kgs Total,15,Kg 000s


In [39]:
from pathlib import Path
out = Path('out')
out.mkdir(exist_ok=True)
new_table.to_csv(out / 'rts_Mass_tidydata.CSV', index = False)